## Assignment 2 - Association Rule Mining

### Overview

For this assignment we are using retail data.
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.  Original data can be downloaded from here http://archive.ics.uci.edu/ml/datasets/online+retail. UCI Machine Learning Repository.
Source:
Dr Daqing Chen, Director: Public Analytics group. chend '@' lsbu.ac.uk, School of Engineering, London South Bank University, London SE1 0AA, UK.

### Algorithm
FPGrowth is an algorithm for discovering frequent itemsets in a transaction database.  FP-growth is a program to find frequent item sets (also closed and maximal as well as generators) with the FP-growth algorithm 

### 1. Data Prep
Before using the FP-Growth algorithm, our .csv data has to be formated in a way in which items are grouped by invoice number.  Transactional data.

In [ ]:
import pandas as pd
df = pd.read_csv('Online Retail.csv', encoding = "ISO-8859-1")
s2 =df.groupby('InvoiceNo')['StockCode'].apply(','.join)
s2 = s2.reset_index()
s2[:5].to_csv('Online Retail.txt', columns = ["StockCode"], header=False, index=False)

### 2. Use Association Rule Mining with FP-Growth

In [2]:
#import libraries
from pyspark.mllib.fpm import FPGrowth
from pyspark import SparkContext
sc = SparkContext()

In [3]:
data = sc.textFile("Online Retail.txt")
transactions = data.map(lambda line: line.strip().split(','))

1) How many frequent itemsets were found with minimum support of .15?  There are 4241 frequent itemsets.

In [4]:
# Set model minSupport = 0.15
model = FPGrowth.train(transactions, minSupport = 0.15, numPartitions = 10)
freqItemsets = model.freqItemsets().collect()

freqItemsetsDict = {}

for i in range(0, len(freqItemsets)):
    freqItemsetsDict[str(freqItemsets[i].items)] = freqItemsets[i].freq

For the number of frequent itemsets:

In [5]:
len(freqItemsets)

4241

2) How many association rules were identified with a minimum confidence of .4?
There are 25035 rules.

In [6]:
# Set minConfidence = 0.4
minConfidence = 0.4
rules = sorted(model._java_model.generateAssociationRules(minConfidence).collect(), 
    key=lambda x: x.confidence(), reverse=True)

for j in range(0, len(rules)):
    antecedent = list(rules[j].antecedent())
    consequent = list(rules[j].consequent())
    ruleItemset = str(antecedent + consequent)
    
    try:
        support = "{0:.2f}".format(freqItemsetsDict[ruleItemset] / transactions.count())
    except KeyError:
        next
    
    confidence = "{0:.2f}".format(rules[j].confidence())
    
    try:
        print(antecedent, '=>', consequent, '(support: '+ str(support), 'confidence: '+ str(confidence) + ')')
    except NameError:
        next

['21754', '22310', '84969', '22623', '22622', '48187"'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"'] => ['21755'] (support: 0.20 confidence: 1.00)
['22749'] => ['22748'] (support: 0.20 confidence: 1.00)
['22749'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22749'] => ['22745'] (support: 0.20 confidence: 1.00)
['22749'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22749'] => ['21755'] (support: 0.20 confidence: 1.00)
['22749'] => ['21777'] (support: 0.20 confidence: 1.00)
['22749'] => ['22310'] (support: 0.20 confidence: 1.00)
['22749'] => ['22622'] (support: 0.20 confidence: 1.00)
['22749'] => ['22623'] (support: 0.20 confidence: 1.00)
['22749'] => ['21754'] (support: 0.20 confidence: 1.00)
['22749'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754',

['21754', '22310', '21777', '22749', '48187"', '22745', '"84879', '21755'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22622', '21777', '22745'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '22748', '22745', '"84879'] => ['48187"'] (support: 0.2

['21754', '22623', '22622', '22749', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622', '22749', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622', '22749', '22748'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622', '22749', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622', '22749', '22748'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22622', '22749', '22748'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22623', '22622', '21777', '"84879', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22623', '22622', '21777', '"84879', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22623', '22622', '21777', '"84879', '21755'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '84969', '22623', '22622', '21777', '"84879', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['

['21754', '22310', '22623', '22622'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '22622'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '22622'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '22622'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '22622'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '22622'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '22622'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '22622'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"', '22745', '"84879', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"', '22745', '"84879', '21755'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22623', '22622', '48187"', '22745', '"84879',

['21754', '84969', '22623', '21777', '22748', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '22749', '22748', '"84879'] => ['21777'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '22749', '22748', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '22749', '22748', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '22749', '22748', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '22749', '22748', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '"84879'] => ['22748'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '"84879'] => ['21777'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '"84879'] => ['22749'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '22622', '"84879'] => ['21755'] (sup

...

['21754', '22622', '21777', '22748', '22745', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22622', '21777', '22748', '22745', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22622', '21777', '22748', '22745', '"84879'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '22622', '21777', '22748', '22745', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['22310'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['21777'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['22749'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['22622'] (support: 0.20 confidence: 1.00)
['22623', '22748'] => ['21754'] (support: 0.20 confidence: 1.00)
['22

['21754', '22310', '84969', '22749'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22749'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22749'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22749'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22749'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22749'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '21777', '22749', '48187"', '"84879', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '21777', '22749', '48187"', '"84879', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '21777', '22749', '48187"', '"84879', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22310', '22623', '21777', '22749', '48187"', '"84879', '21755'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22310

['21754', '22310', '84969', '22622', '21777', '48187"'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22622', '21777', '48187"'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22310', '84969', '22622', '21777', '48187"'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '48187"', '21755'] => ['22310'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '48187"', '21755'] => ['22748'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '48187"', '21755'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '48187"', '21755'] => ['22745'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '48187"', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '48187"', '21755'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '21777', '22749', '48187"', '21755'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22745'] => ['22748'] 

['21754', '22623', '22749', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['48187"'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['22622'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '22623', '22749', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '21777', '22749', '22748', '22745', '21755'] => ['"84879'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '21777', '22749', '22748', '22745', '21755'] => ['48187"'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '21777', '22749', '22748', '22745', '21755'] => ['22622'] (support: 0.20 confidence: 1.00)
['22310', '84969', '22623', '21777', '22749', '22748', '22745', '21755'] => ['21754'] (support: 0.20 confidence: 1.00)
['21777', '22749'

['22623', '48187"', '"84879'] => ['21755'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['22745'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['21754'] (support: 0.20 confidence: 1.00)
['22623', '48187"', '"84879'] => ['84969'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['22623'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['22310'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['"84879'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['22745'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['21755'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['21777'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['22749'] (support: 0.20 confidence: 1.00)
['21754', '22622', '22748', '48187"'] => ['84969'] (support: 0.20 confidence: 1.00)


To view rules:

In [7]:
len(rules)

25035

3) What is the support and confidence of the following association rule: ['21754', '22748'] => ['22745']? Go beyond reporting the values, but include an interpretation of these evaluation metrics.  
  
### Result and interpretation:  
['21754', '22748'] => ['22745'] (support: 0.20 confidence: 1.00)  
Support: 20% of the transactions contain both 21754 and 22748  
Conficence: 100% of the time, Product 22745 occurs in all those transactions which contain 21754 and 22748.